# Python Parsing with NLTK

**(C) 2017-2019 by [Damir Cavar](http://damir.cavar.me/)**

**License:** [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/) ([CA BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/))

This is a tutorial related to the discussion of grammar engineering and parsing in the class *Alternative Syntactic Theories* and *Advanced Natural Language Processing* taught at Indiana University in Spring 2017 and Fall 2018.

## Working with Grammars

The following examples are taken from the NLTK [parsing HOWTO](http://www.nltk.org/howto/parse.html) page.

In [2]:
from nltk import Nonterminal, nonterminals, Production, CFG

In [3]:
nt1 = Nonterminal('NP')
nt2 = Nonterminal('VP')

In [4]:
nt1.symbol()

'NP'

In [5]:
nt1 == Nonterminal('NP')

True

In [6]:
nt1 == nt2

False

In [9]:
S, NP, VP, PP = nonterminals('S, NP, VP, PP')
print(S.symbol())

S


In [10]:
N, V, P, DT = nonterminals('N, V, P, DT')

In [11]:
prod1 = Production(S, [NP, VP])

In [12]:
prod2 = Production(NP, [DT, NP])

In [13]:
prod1.lhs()

S

In [14]:
prod1.rhs()

(NP, VP)

In [15]:
prod1 == Production(S, [NP, VP])

True

In [16]:
prod1 == prod2

False

In [17]:
grammar = CFG.fromstring("""
 S -> NP VP
 PP -> P NP
 PP -> P NP
 NP -> 'the' N | N PP | 'the' N PP
 VP -> V NP | V PP | V NP PP
 N -> 'cat'
 N -> 'fish'
 N -> 'dog'
 N -> 'rug'
 N -> 'mouse'
 V -> 'chased'
 V -> 'sat'
 P -> 'in'
 P -> 'on'
""")

In [18]:
print(grammar)

Grammar with 18 productions (start state = S)
    S -> NP VP
    PP -> P NP
    PP -> P NP
    NP -> 'the' N
    NP -> N PP
    NP -> 'the' N PP
    VP -> V NP
    VP -> V PP
    VP -> V NP PP
    N -> 'cat'
    N -> 'fish'
    N -> 'dog'
    N -> 'rug'
    N -> 'mouse'
    V -> 'chased'
    V -> 'sat'
    P -> 'in'
    P -> 'on'


## Feature Structures

One can build complex feature structures using the following strategies:

In [21]:
import nltk

fstr = nltk.FeatStruct("[POS='N', AGR=[PER=3, NUM='pl', GND='fem']]")
print(fstr)

[       [ GND = 'fem' ] ]
[ AGR = [ NUM = 'pl'  ] ]
[       [ PER = 3     ] ]
[                       ]
[ POS = 'N'             ]


Creating shared paths is also possible:

In [22]:
fstr2 = nltk.FeatStruct("""[NAME='Lee', ADDRESS=(1)[NUMBER=74, STREET='rue Pascal'],
                          SPOUSE=[NAME='Kim', ADDRESS->(1)]]""")
print(fstr2)

[ ADDRESS = (1) [ NUMBER = 74           ] ]
[               [ STREET = 'rue Pascal' ] ]
[                                         ]
[ NAME    = 'Lee'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'Kim' ]           ]


Let us create feature structures and try out unification:

In [25]:
fs1 = nltk.FeatStruct("[AGR=[PER=3, NUM='pl', GND='fem'], POS='N']")
fs2 = nltk.FeatStruct("[POS='N', AGR=[PER=3, GND='fem']]")

print(fs1.unify(fs2))

[       [ GND = 'fem' ] ]
[ AGR = [ NUM = 'pl'  ] ]
[       [ PER = 3     ] ]
[                       ]
[ POS = 'N'             ]


## Chart Parser

The following examples are taken from the NLTK [parsing HOWTO](http://www.nltk.org/howto/parse.html) page.

In [26]:
import nltk

In [27]:
nltk.parse.chart.demo(2, print_times=False, trace=1,
                       sent='I saw a dog', numparses=1)

* Sentence:
I saw a dog
['I', 'saw', 'a', 'dog']

* Strategy: Bottom-up

|.    I    .   saw   .    a    .   dog   .|
|[---------]         .         .         .| [0:1] 'I'
|.         [---------]         .         .| [1:2] 'saw'
|.         .         [---------]         .| [2:3] 'a'
|.         .         .         [---------]| [3:4] 'dog'
|>         .         .         .         .| [0:0] NP -> * 'I'
|[---------]         .         .         .| [0:1] NP -> 'I' *
|>         .         .         .         .| [0:0] S  -> * NP VP
|>         .         .         .         .| [0:0] NP -> * NP PP
|[--------->         .         .         .| [0:1] S  -> NP * VP
|[--------->         .         .         .| [0:1] NP -> NP * PP
|.         >         .         .         .| [1:1] Verb -> * 'saw'
|.         [---------]         .         .| [1:2] Verb -> 'saw' *
|.         >         .         .         .| [1:1] VP -> * Verb NP
|.         >         .         .         .| [1:1] VP -> * Verb
|.         [--------->

This is an example how to apply top-down parsing:

In [34]:
nltk.parse.chart.demo(1, print_times=True, trace=0,
                       sent='I saw John with Mary', numparses=2)

* Sentence:
I saw John with Mary
['I', 'saw', 'John', 'with', 'Mary']

* Strategy: Top-down



ValueError: Grammar does not cover some of the input words: "'Mary'".

This is how to apply bottom-up parsing:

In [35]:
nltk.parse.chart.demo(2, print_times=False, trace=0,
                       sent='I saw John with a dog', numparses=2)

* Sentence:
I saw John with a dog
['I', 'saw', 'John', 'with', 'a', 'dog']

* Strategy: Bottom-up

Nr edges in chart: 53
(S
  (NP I)
  (VP (VP (Verb saw) (NP John)) (PP with (NP (Det a) (Noun dog)))))
(S
  (NP I)
  (VP (Verb saw) (NP (NP John) (PP with (NP (Det a) (Noun dog))))))



In [36]:
nltk.parse.featurechart.demo(print_times=False,
                              print_grammar=True,
                              parser=nltk.parse.featurechart.FeatureChartParser,
                              sent='I saw John with a dog')


Grammar with 18 productions (start state = S[])
    S[] -> NP[] VP[]
    PP[] -> Prep[] NP[]
    NP[] -> NP[] PP[]
    VP[] -> VP[] PP[]
    VP[] -> Verb[] NP[]
    VP[] -> Verb[]
    NP[] -> Det[pl=?x] Noun[pl=?x]
    NP[] -> 'John'
    NP[] -> 'I'
    Det[] -> 'the'
    Det[] -> 'my'
    Det[-pl] -> 'a'
    Noun[-pl] -> 'dog'
    Noun[-pl] -> 'cookie'
    Verb[] -> 'ate'
    Verb[] -> 'saw'
    Prep[] -> 'with'
    Prep[] -> 'under'

* FeatureChartParser
Sentence: I saw John with a dog
|.I.s.J.w.a.d.|
|[-] . . . . .| [0:1] 'I'
|. [-] . . . .| [1:2] 'saw'
|. . [-] . . .| [2:3] 'John'
|. . . [-] . .| [3:4] 'with'
|. . . . [-] .| [4:5] 'a'
|. . . . . [-]| [5:6] 'dog'
|[-] . . . . .| [0:1] NP[] -> 'I' *
|[-> . . . . .| [0:1] S[] -> NP[] * VP[] {}
|[-> . . . . .| [0:1] NP[] -> NP[] * PP[] {}
|. [-] . . . .| [1:2] Verb[] -> 'saw' *
|. [-> . . . .| [1:2] VP[] -> Verb[] * NP[] {}
|. [-] . . . .| [1:2] VP[] -> Verb[] *
|. [-> . . . .| [1:2] VP[] -> VP[] * PP[] {}
|[---] . . . .| [0:2] S[] ->

## Loading grammars from files and editing them

We will need the following NLTK modules in this section:

In [37]:
from nltk import CFG
from nltk.grammar import FeatureGrammar as FCFG

We can load a grammar from a file, that is located in the same folder as the current Jupyter notebook, in the following way:

In [38]:
cfg = nltk.data.load('spanish1.cfg')

We instantiate a ChartParser object with this grammar:

In [39]:
cp1 = nltk.parse.ChartParser(cfg)

The *ChartParser* object has a parse-function that takes a list of tokens as a parameter. The token list can be generated using a language specific tokenizer. In this case we simply tokenize using the Python-function *split* on strings. The output of the parse function is a list of trees. We loop through the list of parse trees and print them out:

In [40]:
for x in cp1.parse("las mujeres adoran el Lucas".split()):
    print(x)

(S
  (SN (det las) (GN (nom_com mujeres)))
  (SV (v adoran) (SN (det el) (GN (nom_prop Lucas)))))


We can also edit a grammar directly:

In [ ]:
cfg2 = CFG.fromstring("""
 S -> NP VP
 PP -> P NP
 NP -> 'the' N | N PP | 'the' N PP
 VP -> V NP | V PP | V NP PP
 N -> 'cat'
 N -> 'dog'
 N -> 'bird'
 N -> 'rug'
 V -> 'chased'
 V -> 'sat'
 V -> 'bit'
 P -> 'in'
 P -> 'on'
""")

We parse our example sentences using the same approach as above:

In [ ]:
cp2 = nltk.parse.ChartParser(cfg2)
for x in cp2.parse("the bird bit the dog".split()):
    print(x)

The previous example included a Context-free grammar. In the following example we load a Context-free Grammar with Features, instantiate a *FeatureChartParser*, and loop through the parse trees that are generated by our grammar to print them out:

In [ ]:
fcfg = nltk.data.load('spanish1.fcfg')
fcp1 = nltk.parse.FeatureChartParser(fcfg)
for x in fcp1.parse(u"Miguel adoró el gato".split()):
    print(x)

We can edit a Feature CFG in the same way directly in this notebook and then parse with it:

In [ ]:
fcfg2 = FCFG.fromstring("""
% start CP
# ############################
# Grammar Rules
# ############################
CP -> Cbar[stype=decl]
Cbar[stype=decl] -> IP[+TNS]
IP[+TNS] -> DP[num=?n,pers=?p,case=nom] VP[num=?n,pers=?p]
DP[num=?n,pers=?p,case=?k] ->  Dbar[num=?n,pers=?p,case=?k]
Dbar[num=?n,pers=?p] -> D[num=?n,DEF=?d,COUNT=?c] NP[num=?n,pers=?p,DEF=?d,COUNT=?c]
Dbar[num=?n,pers=?p] -> NP[num=?n,pers=?p,DEF=?d,COUNT=?c]
Dbar[num=?n,pers=?p,case=?k] -> D[num=?n,pers=?p,+DEF,type=pron,case=?k]
NP[num=?n,pers=?p,COUNT=?c] -> N[num=?n,pers=?p,type=prop,COUNT=?c]
VP[num=?n,pers=?p] -> V[num=?n,pers=?p,val=1]
VP[num=?n,pers=?p] -> V[num=?n,pers=?p,val=2] DP[case=acc]
#
# ############################
# Lexical Rules
# ############################
D[-DEF,+COUNT,num=sg] -> 'a'
D[-DEF,+COUNT,num=sg] -> 'an'
D[+DEF] -> 'the'
D[+DEF,gen=f,num=sg,case=nom,type=pron] -> 'she'
D[+DEF,gen=m,num=sg,case=nom,type=pron] -> 'he'
D[+DEF,gen=n,num=sg,type=pron] -> 'it'
D[+DEF,gen=f,num=sg,case=acc,type=pron] -> 'her'
D[+DEF,gen=m,num=sg,case=acc,type=pron] -> 'him'
N[num=sg,pers=3,type=prop] -> 'John' | 'Sara' | 'Mary'
V[tns=pres,num=sg,pers=3,val=2] -> 'loves' | 'calls' | 'sees' | 'buys'
N[num=sg,pers=3,-COUNT] -> 'furniture' | 'air' | 'justice'
N[num=sg,pers=3] -> 'cat' | 'dog' | 'mouse'
N[num=pl,pers=3] -> 'cats' | 'dogs' | 'mice'
V[tns=pres,num=sg,pers=3,val=1] -> 'sleeps' | 'snores'
V[tns=pres,num=sg,pers=1,val=1] -> 'sleep' | 'snore'
V[tns=pres,num=sg,pers=2,val=1] -> 'sleep' | 'snore'
V[tns=pres,num=pl,val=1] -> 'sleep' | 'snore'
V[tns=past,val=1] -> 'slept' | 'snored'
V[tns=pres,num=sg,pers=3,val=2] -> 'calls' | 'sees' | 'loves'
V[tns=pres,num=sg,pers=1,val=2] -> 'call' | 'see' | 'love'
V[tns=pres,num=sg,pers=2,val=2] -> 'call' | 'see' | 'love'
V[tns=pres,num=pl,val=2] -> 'call' | 'see' | 'love'
V[tns=past,val=2] -> 'called' | 'saw' | 'loved'
""")

We can now create a parser instance and parse with this grammar:

In [ ]:
fcp2 = nltk.parse.FeatureChartParser(fcfg2, trace=1)
sentence = "John buys him"
result = list(fcp2.parse(sentence.split()))
if result:
    for x in result:
        print(x)
else:
    print("*", sentence)

Countable nouns and articles in a DP:

DPs and pronouns

CP/IP sentence structures

## Different Parsers

This is a list of the different Feature Parsers in NLTK.

- nltk.parse.featurechart.FeatureChartParser
- nltk.parse.featurechart.FeatureTopDownChartParser
- nltk.parse.featurechart.FeatureBottomUpChartParser
- nltk.parse.featurechart.FeatureBottomUpLeftCornerChartParser
- nltk.parse.earleychart.FeatureIncrementalChartParser
- nltk.parse.earleychart.FeatureEarleyChartParser
- nltk.parse.earleychart.FeatureIncrementalTopDownChartParser
- nltk.parse.earleychart.FeatureIncrementalBottomUpChartParser
- nltk.parse.earleychart.FeatureIncrementalBottomUpLeftCornerChartParser

I do not know whether this is an exhaustive list.

(C) 2017-2018 by [Damir Cavar](http://damir.cavar.me/) - [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/) ([CA BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/))